In [15]:
ball_positions = [
    {1: [100, 150, 110, 160]},  # Frame 1: Ball detected
    {1: [102, 152, 112, 162]},  # Frame 2: Ball detected
    {},                         # Frame 3: Ball not detected (missing)
    {1: [106, 156, 116, 166]},  # Frame 4: Ball detected
]
ball_positions


[{1: [100, 150, 110, 160]},
 {1: [102, 152, 112, 162]},
 {},
 {1: [106, 156, 116, 166]}]

In [9]:
ball_positions = [x.get(1, []) for x in ball_positions]
ball_positions


[[100, 150, 110, 160], [102, 152, 112, 162], [], [106, 156, 116, 166]]

In [11]:
import pandas as pd
df_ball_positions = pd.DataFrame(ball_positions, columns=['x1', 'y1', 'x2', 'y2'])
df_ball_positions.head()

,x1,y1,x2,y2
0,100.0,150.0,110.0,160.0
1,102.0,152.0,112.0,162.0
2,NaN,NaN,NaN,NaN
3,106.0,156.0,116.0,166.0


In [13]:
df_ball_positions = df_ball_positions.interpolate()
df_ball_positions = df_ball_positions.bfill() #ensures that any remaining NaN values after interpolation are filled using the next valid detected ball position from subsequent frames. This ensures there are no gaps in the ball positions.
df_ball_positions.head()


,x1,y1,x2,y2
0,100.0,150.0,110.0,160.0
1,102.0,152.0,112.0,162.0
2,104.0,154.0,114.0,164.0
3,106.0,156.0,116.0,166.0


In [14]:
ball_positions = [{1: x} for x in df_ball_positions.to_numpy().tolist()]
ball_positions

[{1: [100.0, 150.0, 110.0, 160.0]},
 {1: [102.0, 152.0, 112.0, 162.0]},
 {1: [104.0, 154.0, 114.0, 164.0]},
 {1: [106.0, 156.0, 116.0, 166.0]}]

In [19]:
data = {'x1': [100, 102, None, None],
        'y1': [150, 152, None, None],
        'x2': [110, 112, None, None],
        'y2': [160, 162, None, None]}

df = pd.DataFrame(data)
df = df.interpolate()
df = df.bfill()
df.head()


,x1,y1,x2,y2
0,100.0,150.0,110.0,160.0
1,102.0,152.0,112.0,162.0
2,102.0,152.0,112.0,162.0
3,102.0,152.0,112.0,162.0


: 

In [12]:
a = [{1: [100.0, 150.0, 110.0, 160.0]},
 {1: [102.0, 152.0, 112.0, 162.0]},
 {1: [104.0, 154.0, 114.0, 164.0]},
 {1: [106.0, 156.0, 116.0, 166.0]}]

In [13]:
s = [{0: [100.0, 150.0, 110.0, 160.0]},
 {1: [102.0, 152.0, 112.0, 162.0]}]

In [7]:
a[0][1]

[100.0, 150.0, 110.0, 160.0]

In [9]:
import cv2
import numpy as np
import sys
sys.path.append('../')
import constants
from utils import ( convert_pixel_distance_to_meters , 
                   convert_meters_to_pixel_distance , 
                   get_foot_position , 
                   get_closest_keypoint_index ,
                   get_height_of_bbox , 
                   measure_xy_distance ,
                   get_center_of_bbox ,
                   measure_distance)

In [29]:
from utils import (read_video, save_video)
from trackers import PlayerTracker , BallTracker
from court_line_detector import CourtLineDetector
from mini_court import MiniCourt
import cv2 

player_tracker = PlayerTracker(model_path="yolov8x")
ball_tracker = BallTracker(model_path="models/yolo5_best.pt")

input_video_path = 'input_videos/input_video.mp4'
video_frames = read_video(input_video_path)

court_model_path = "models/keypoints_model.pth"
court_line_detector = CourtLineDetector(court_model_path)
court_keypoints = court_line_detector.predict(video_frames[0])



player_detections = player_tracker.detect_frames(video_frames , read_from_stub=True , 
                                                     stub_path="tracker_stubs/player_detections.pkl")
    
ball_detections = ball_tracker.detect_frames(video_frames , read_from_stub=True , 
                                                     stub_path="tracker_stubs/ball_detections.pkl")

player_detections = player_tracker.choose_and_filter_players(court_keypoints ,player_detections)
ball_detections = ball_tracker.interpolate_ball_positions(ball_detections)
print("player_detections",player_detections[0])
print("ball_detections",ball_detections[0])

player_heights = {
            1: constants.PLAYER_1_HEIGHT_METERS,
            2: constants.PLAYER_2_HEIGHT_METERS
        }
output_player_boxes = []
output_ball_boxes = []

for frame_num , player_bbox in enumerate(player_detections):
        print('frame_num' , frame_num)
        print('player key', player_bbox.keys())
        ball_box = ball_detections[frame_num][1]
        ball_position = get_center_of_bbox(ball_box)
        print('ball_position' , ball_position)
        closest_player_id_to_ball = min(player_bbox.keys(), key=lambda x: measure_distance(ball_position, get_center_of_bbox(player_bbox[x]))) # anonymous function that takes x (which is the player ID, 1 or 2) as an argument and return the id that have the min distance
        print("closest_player_id_to_ball",closest_player_id_to_ball)

        output_player_bboxes_dict = {}
        for player_id , bbox in player_bbox.items():
            foot_position = get_foot_position(bbox)

            # get the cloesest keypoint index
            closest_key_point_index = get_closest_keypoint_index(foot_position, court_keypoints , [0,2,12,13])
            print("closest_key_point_index",closest_key_point_index)
            
            closest_key_point = (court_keypoints[closest_key_point_index*2], 
                                     court_keypoints[closest_key_point_index*2+1])

                # Get Player height in pixels
            frame_index_min = max(0, frame_num-20)
            frame_index_max = min(len(player_detections), frame_num+50)
            bboxes_heights_in_pixels = [get_height_of_bbox(player_detections[i][player_id]) for i in range (frame_index_min,frame_index_max)]

c:\Users\RTC\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\RTC\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


player_detections {1: [480.2823486328125, 751.5975341796875, 624.8380126953125, 930.6015625], 2: [1030.5684814453125, 201.68310546875, 1095.9012451171875, 308.51904296875]}
ball_detections {1: [892.924560546875, 616.61474609375, 911.2674560546875, 636.578369140625]}
frame_num 0
player key dict_keys([1, 2])
ball_position (902, 626)
closest_player_id_to_ball 2
closest_key_point_index 2
closest_key_point_index 0
frame_num 1
player key dict_keys([1, 2])
ball_position (902, 626)
closest_player_id_to_ball 2
closest_key_point_index 2
closest_key_point_index 0
frame_num 2
player key dict_keys([1, 2])
ball_position (887, 640)
closest_player_id_to_ball 1
closest_key_point_index 2
closest_key_point_index 0
frame_num 3
player key dict_keys([1, 2])
ball_position (873, 654)
closest_player_id_to_ball 1
closest_key_point_index 2
closest_key_point_index 0
frame_num 4
player key dict_keys([1, 2])
ball_position (858, 668)
closest_player_id_to_ball 1
closest_key_point_index 2
closest_key_point_index 0
fra

In [30]:
len(player_detections)

214